In [1]:
"""
snipper.io/s/-MSTCnlcnC3p7aUSnOm_
https://www.kaggle.com/c/clickbait-news-detection/

"""

'\nsnipper.io/s/-MSTCnlcnC3p7aUSnOm_\nhttps://www.kaggle.com/c/clickbait-news-detection/\n\n'

In [32]:
from copy import deepcopy

from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, NGram, HashingTF, IDF, CountVectorizer, OneHotEncoder, StringIndexer, Word2Vec
from pyspark.sql.functions import col, udf, length, avg, lit, concat, size, array
from pyspark.sql.types import IntegerType, StringType, ArrayType
#from pyspark.mllib.classification import NaiveBayes#, NaiveBayesModel
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql import SparkSession

import re
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
#import nltk
#nltk.download('wordnet')

In [3]:
try:
    sc.stop()
except:
    print("no sc to stop")
    
# try:
#     spark.stop()
# except Exception as e:
#     print("{}, {}".format(e,type(e)))
    

In [4]:
sc

AttributeError: 'NoneType' object has no attribute 'sc'

<SparkContext master=local[*] appName=PySparkShell>

In [5]:
spark = SparkSession.builder.appName('ClickBait')\
    .getOrCreate()

In [6]:
spark

In [7]:
# pyspark_stopwords = StopWordsRemover().getStopWords()#https://spark.apache.org/docs/2.2.0/api/python/pyspark.ml.html#pyspark.ml.feature.StopWordsRemover
# display(type(pyspark_stopwords), len(pyspark_stopwords), pyspark_stopwords)

In [8]:
train_path = "file:///home/hadoop/data/clickbait/train.csv"
#train_path = "/data/train.csv"
#train_path = "hdfs://localhost:9000/data/train.csv;"


In [9]:
# load main training dataframe

t_df = spark.read.csv(train_path, header=True, inferSchema=True, encoding="utf-8").limit(5000)
t_df.persist()

DataFrame[id: string, title: string, text: string, label: string]

In [10]:
t_df.count()

5000

In [11]:
t_df=t_df.na.drop()

t_df.persist()

DataFrame[id: string, title: string, text: string, label: string]

In [12]:
t_df.count()

4965

In [13]:
#from string import punctuation

def strip_possessive(l_):
    ptrn=re.compile(r"['`’]s", flags=re.I)
    r__ = [ptrn.sub('', s_) for s_ in l_]
    return(r__)  

@udf("string")
def strip_possessive_str(s_):
    ptrn=re.compile(r"['`’]s", flags=re.I)
    r__ = ptrn.sub(' ', s_)
    return(r__)


def strip_func(l_):
    ptrn = re.compile('[\W_]+')
    r__ = [ptrn.sub('', s_) for s_ in l_]
    return(r__)

@udf("string")
def strip_func_str(s_):
    ptrn = re.compile('[\W_]+')
    r__ = ptrn.sub(' ', s_)
    return(r__)

#udf_strip_func = udf(strip_func, ArrayType(StringType()))

# def strip_pos_spchars(l_):
#     ptrn_pos = re.compile(r"[']s", flags=re.I)
#     ptrn_spchars = re.compile('[\W_]+')
#     r__ = [ptrn.sub('', s_) for s_ in l_]
#     [ptrn_spchars.sub('', ptrn_pos.sub('', s_)) for s_ in l_]
#     return(r__)     
#x = list(one_sss_list.take(1)[0].asDict()['sss'])

#x = ["World's fair", "Biden's administration", "fergi's song", "Xi's policy"]

#x, strip_func(x), strip_possessive(x), #strip_possessive_str("World's fair ... Biden's administration, Fergi's song. Xi's policy")
#strip_func(one_sss_list.take(1)[0].asDict()['sss'])

In [14]:
help(udf)

Help on function udf in module pyspark.sql.functions:

udf(f=None, returnType=StringType)
    Creates a user defined function (UDF).
    
    .. note:: The user-defined functions are considered deterministic by default. Due to
        optimization, duplicate invocations may be eliminated or the function may even be invoked
        more times than it is present in the query. If your function is not deterministic, call
        `asNondeterministic` on the user defined function. E.g.:
    
    >>> from pyspark.sql.types import IntegerType
    >>> import random
    >>> random_udf = udf(lambda: int(random.random() * 100), IntegerType()).asNondeterministic()
    
    .. note:: The user-defined functions do not support conditional expressions or short circuiting
        in boolean expressions and it ends up with being executed all internally. If the functions
        can fail on special rows, the workaround is to incorporate the condition into the functions.
    
    .. note:: The user-defined

In [57]:
stemmer=SnowballStemmer("english")

lemmatizer = WordNetLemmatizer()

# def stem_lem(w_, pos='n'):
#     return(lemmatizer.lemmatize(stemmer.stem(w_), pos=pos))

# stem_lem_cases= ["writing", "write", "wrote", "writing", "written", "drag", "dragged", "drug", "dragon"]

# display(
#     stem_lem_cases,
#     [stemmer.stem(w) for w in stem_lem_cases],
#     [lemmatizer.lemmatize(w, pos='n') for w in stem_lem_cases],
#     [stem_lem(w) for w in stem_lem_cases]
#     )

#stem_lem = udf(stem_lem, StringType())

@udf(ArrayType(StringType()))
def stem_lem(l_, pos='n'):
    return([lemmatizer.lemmatize(stemmer.stem(w_), pos=pos) for w_ in l_])
    

In [42]:
t_df = t_df.withColumn("s", concat(col('title'), col('text')))

In [17]:
t_df.select('s').limit(20).show()

+--------------------+
|                   s|
+--------------------+
|China and Economi...|
|Trade to Be a Big...|
|The Top Beaches I...|
|Sheriff’s Report ...|
|Surgeon claiming ...|
|This Is How Diffe...|
|Samantha Bee and ...|
|Krauthammer: Syri...|
|Rust Belt voters ...|
|As Illegal Outpos...|
|$94.4m $198m $112...|
|Trump’s Mar-a-Lag...|
|STOCKS SLIP TO EN...|
|Jared Kushner in ...|
|He Cheated. Now H...|
|The fabulous life...|
|Destinations Insi...|
|President Donald ...|
|4 Things That You...|
|Freddie Flintoff ...|
+--------------------+



In [18]:
t_df = t_df.withColumn("s", strip_func_str(strip_possessive_str(col('s'))))


t_df.select('s').limit(5).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
"’"=="`"

False

In [20]:
selection = ['s']

t_df \
    .select(*(length(col(c)).alias(c) for c in selection)) \
    .agg(*(avg(col(c)).alias(c) for c in selection)).show()



+------------------+
|                 s|
+------------------+
|2423.5222557905336|
+------------------+



In [21]:
from collections.abc import Iterable

def get_avg_len_cols(df, selection):
    assert isinstance(selection, Iterable)
    return(
        df \
        .select(*(length(col(c)).alias(c) for c in selection)) \
        .agg(*(avg(col(c)).alias(c) for c in selection)).show()
              )

def get_avg_size_cols(df, selection):
    assert isinstance(selection, Iterable)
    return(
        df \
        .select(*(size(col(c)).alias(c) for c in selection)) \
        .agg(*(avg(col(c)).alias(c) for c in selection)).show()
              )

In [22]:
t_df = Tokenizer(inputCol = "s", outputCol = "ss").transform(t_df)

"""
some punctuation like ',' is still showing after Tokenizer().transform.
"""

"\nsome punctuation like ',' is still showing after Tokenizer().transform.\n"

In [23]:
t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- ss: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [35]:
t_df.select("ss").limit(5).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [36]:
get_avg_size_cols(t_df, ['ss'])

+-----------------+
|               ss|
+-----------------+
|425.5635448136959|
+-----------------+



In [25]:
t_df = StopWordsRemover(inputCol = "ss", outputCol="sss").transform(t_df)

In [37]:
get_avg_size_cols(t_df, ['sss'])

+------------------+
|               sss|
+------------------+
|250.11701913393756|
+------------------+



In [27]:
t_df.select('sss').limit(5).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
t_df.sample(False, 0.1, seed=0).limit(1).sss.getItem

<bound method Column.getItem of Column<b'sss'>>

In [29]:
t_df.sample(False, 0.1, seed=0).select("sss").limit(1).show(truncate=False)

one_sss_list=t_df.sample(False, 0.1, seed=0).select("sss").limit(1)

+--------------------------------------------------------------------------------------------------+
|sss                                                                                               |
+--------------------------------------------------------------------------------------------------+
|[, asap, ferg, remy, ma, rep, east, coast, new, song, asap, ferg, repping, east, coast, new, song]|
+--------------------------------------------------------------------------------------------------+



In [58]:
# #dir(one_sss_list),
# dir(one_sss_list.take(1)[0])
# one_sss_list.take(1)[0].asDict()['sss']
stem_lem(t_df.sample(False, 0.1, seed=0).select("sss").limit(1))

TypeError: Invalid argument, not a string or column: DataFrame[sss: array<string>] of type <class 'pyspark.sql.dataframe.DataFrame'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [59]:
t_df = t_df.withColumn("ssss", stem_lem(array(col("sss"))))
#t_df.persist()

In [60]:
t_df.select('sss', "ssss").limit(5).show(truncate=False)

Py4JJavaError: An error occurred while calling o730.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 58.0 failed 1 times, most recent failure: Lost task 0.0 in stage 58.0 (TID 39, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 352, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 142, in dump_stream
    for obj in iterator:
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 341, in _batched
    for item in iterator:
  

In [ ]:
# Preprocessing Strategy
"""
plan is to turn this into a Pipeline([Tokenizer,StopWordsRemover,Ngram1,Ngram2...])
"""
raise AssertionError('stop')

#count_tokens = udf(lambda words : len(words) , IntegerType())
text_cols= ["text","title"]
for t_c in text_cols:
    t_df = Tokenizer(inputCol = t_c, outputCol = "token_"+t_c).transform(t_df)
    t_df = StopWordsRemover(inputCol = "token_"+t_c, outputCol = t_c+"_f_token").transform(t_df)
    
    for i in range(1,4):
        t_df= NGram(n=i, inputCol=t_c+"_f_token", outputCol=t_c+"_ngram_"+str(i)).transform(t_df)
        
        #CountVectorizer for each ngram col
#         c_v = CountVectorizer(inputCol= t_c+"_ngram_"+str(i), outputCol=t_c+"_ngram_"+str(i)+"cvfeat",
#                               minDF=2.0, vocabSize=2**14)
        
        #indexer = StringIndexer(inputCol=t_c+"_ngram_"+str(i), outputCol=t_c+"_ngram_"+str(i)+"ix_feat")
        #display(t_c+"_ngram_"+str(i), t_c+"_ngram_"+str(i)+"cvfeat")
        
        #m_ = c_v.fit(t_df) # c_v.fit is failing on 
        #t_df = m_.tranform(t_df)
        #m_ = c_v.fit(t_df)
        #t_df = m_.transform(t_df)
    
    t_df = t_df.drop("token_"+t_c)



t_df.persist()

In [ ]:
t_df.printSchema()

In [ ]:
# # Preprocessing Strategy
# text_cols= ["text","title"]

# for t_c in text_cols:
#     t_df = Tokenizer(inputCol = t_c, outputCol = "token_"+t_c).transform(t_df)
#     t_df = StopWordsRemover(inputCol = "token_"+t_c, outputCol = t_c+"_f_token").transform(t_df)
    
#     for i in range(1,4):
#         t_df= NGram(n=i, inputCol=t_c+"_f_token", outputCol=t_c+"_ngram_"+str(i)).transform(t_df)
        
#         #CountVectorizer for each ngram col
#         c_v = CountVectorizer(inputCol= t_c+"_ngram_"+str(i), outputCol=t_c+"_ngram_"+str(i)+"cvfeat",
#                               minDF=2.0, vocabSize=2**14)
        
#         #indexer = StringIndexer(inputCol=t_c+"_ngram_"+str(i), outputCol=t_c+"_ngram_"+str(i)+"ix_feat")
#         #display(t_c+"_ngram_"+str(i), t_c+"_ngram_"+str(i)+"cvfeat")
        
#         #m_ = c_v.fit(t_df) # c_v.fit is failing
#         #t_df = m_.tranform(t_df)
        

#     t_df = t_df.drop("token_"+t_c)


# #t_df.show()


# # def build_ngrams_wocs(inputCol=["text","target"], n=3):
# #     tokenizer = [Tokenizer(inputCol="text", outputCol="words")]
# #     ngrams = [
# #         NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))
# #         for i in range(1, n + 1)
# #     ]

# #     cv = [
# #         CountVectorizer(vocabSize=5460,inputCol="{0}_grams".format(i),
# #             outputCol="{0}_tf".format(i))
# #         for i in range(1, n + 1)
# #     ]
# #     idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]

# #     assembler = [VectorAssembler(
# #         inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],
# #         outputCol="features"
# #     )]
# #     label_stringIdx = [StringIndexer(inputCol = "target", outputCol = "label")]
# #     lr = [LogisticRegression(maxIter=100)]
# #     return Pipeline(stages=tokenizer + ngrams + cv + idf+ assembler + label_stringIdx+lr)



In [ ]:
#t_df.select("title_ngram_3").show(5, truncate=False)

t_df.select("text_ngram_1").limit(1).collect()[0]['text_ngram_1']

{type(z) for z in t_df.select("text_ngram_1").limit(1).collect()[0]['text_ngram_1']}
#, t_df.columns

In [ ]:
t_df.printSchema()

In [ ]:
c_v = CountVectorizer(inputCol="text_ngram_2", outputCol="999998",
                     minDF=2.0, vocabSize=2**14)

# _p = {'inputCol':"title_ngram_1", 'outputCol':"title_ngram_1_cv",
#       'minDF':2.0, 'vocabSize':2**14}

#m_ = c_v.fit(t_df, params={'inputCol': 'text_ngram_1'})
mdl = c_v.fit(t_df)

t_df=mdl.transform(t_df)
t_df.persist()

In [ ]:
# df = spark.createDataFrame(
#     [(0, ["a", "b a", "c"]), (1, ["a", "b", "b", "c", "a"])],
#     ["label", "raw"])
# cv = CountVectorizer(inputCol="raw", outputCol="vectors")
# model = cv.fit(df)
# model.transform(df).show(truncate=False)

In [ ]:
t_df.printSchema(), t_df.columns

In [ ]:
#text_tokenizer = Tokenizer(inputCol = 'text', outputCol = 'text_words')
#ttl_tokenizer = Tokenizer(inputCol = 'title', outputCol = 'ttl_words')
#regex_tokenizer = RegexTokenizer(inputCol= 'sentence', outputCol= 'words', pattern='\\W')
#t_df = tokenizer.transform(t_df)
# t_df = text_tokenizer.transform(ttl_tokenizer.transform(t_df))
help(CountVectorizer)

In [ ]:
## Feature building

fcols = [
#  'text_f_token',
  'text_ngram_1',
  'text_ngram_2',
  'text_ngram_3',
#  'title_f_token',
  'title_ngram_1',
  'title_ngram_2',
  'title_ngram_3']

indexer = StringIndexer(inputCol="label", outputCol="label_index")
t_df = indexer.fit(t_df).transform(t_df)

cv = CountVectorizer(inputCol="text_ngram_2", outputCol="features")
t_df=cv.fit(t_df).transform(t_df)

t_df.persist()



# ohe = OneHotEncoderEstimator(
#     inputCols= fcols,
#     outputCols=["feat_"+s for s in fcols])

#f_df=ohe.fit(t_df).transform(t_df)
    
#f_df=cv.fit(t_df).transform(t_df)
#OneHotEncoderEstimator
#hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [ ]:
# idf= IDF(inputCol='rawFeatures', outputCol='features')
# idf_model = idf.fit(featurized_df)
# rescaled_df = idf_model.transform(featurized_df)

# rescaled_df.show(truncate=False)

t_df.printSchema()

In [ ]:
help(NaiveBayes)

In [ ]:
train, test = t_df.randomSplit([.8,.2], seed = 999)

In [ ]:
NB = NaiveBayes(modelType= "multinomial", labelCol="label_index", featuresCol ="999998")

In [ ]:
nb_model= NB.fit(train)
nb_predictions= nb_model.transform(test)



"""
Py4JJavaError: An error occurred while calling o459.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 23, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
"""

In [ ]:

evaluator = MulticlassClassificationEvaluator(labelCol= "label_index", predictionCol = "prediction", 
                                              metricName="accuracy")

nb_accuracy = evaluator.evaluate(nb_predictions)

print("Accuracy : ", nb_accuracy)

In [ ]:
"""
https://spark.apache.org/docs/2.2.0/ml-pipeline.html

Plan:
    - use Pipeline([transformer_n, estimator_n...])
    - Improve preprocessing (punctuation, special characters, stemming)
    - Include transformers and estimators for each n_gram length

"""